In [1]:
import open3d
import argparse
import sys
sys.path.append('.')
import os
import time
import h5py
import datetime
import numpy as np
from matplotlib import pyplot as plt
import torch
import torch.nn.parallel
import torch.utils.data
from torch.utils.data import DataLoader
import torch.nn.functional as F
from data_utils.ModelNetDataLoader import ModelNetDataLoader, load_data, class_names
from pathlib import Path
from tqdm import tqdm
from utils import test, save_checkpoint, select_avaliable, mkdir
import log
from model.pointnet2 import PointNet2ClsMsg
from model.pointnet import PointNetCls, feature_transform_reguliarzer
import colorsys
from clf import parse_args, root
from adv.chamfer import chamfer_batch

args = parse_args([])
args.pretrain = 'experiment/weights/clf-pointnet-0.89730-0076.pth'
args.batch_size = 8
test_data, test_label = load_data(root, train = False)
testDataset = ModelNetDataLoader(test_data, test_label)
testDataLoader = torch.utils.data.DataLoader(testDataset, batch_size=args.batch_size, shuffle=False)

log.debug('Building Model',args.model_name)
if args.model_name == 'pointnet':
    num_class = 40
    model = PointNetCls(num_class,args.feature_transform).cuda()  
else:
    model = PointNet2ClsMsg().cuda()

if args.pretrain is None:
    log.err('No pretrain model')
else:
    log.debug('Loading pretrain model...')
    checkpoint = torch.load(args.pretrain)
    model.load_state_dict(checkpoint)
    model.eval()

job: clf model_name: pointnet 
pretrain: experiment/weights/clf-pointnet-0.89730-0076.pth 
batch_size: 16 gpu_count: 1 
Building Model pointnet 
Loading pretrain model... 


In [3]:
def num(x):
    return x.detach().cpu().numpy()

def disp_pt_cloud(x,channal_first = True):
    cloud = open3d.geometry.PointCloud()
    cloud.points = open3d.utility.Vector3dVector(x)

    vis = open3d.visualization.VisualizerWithKeyCallback()
    vis.create_window(args.model_name, height=800, width=800, left=200, top=0)
    opt = vis.get_render_option().background_color = np.asarray([0, 0, 0])
    vis.add_geometry(cloud)
    vis.register_key_callback(32, lambda vis: exit())
    vis.run()
    vis.destroy_window()

for eps in np.linspace(0,0.1,num=20):
    for pts, gt in testDataLoader:
        gt = gt[:, 0].long().cuda()
        pts = pts.transpose(2, 1).cuda() # make channal first
        pts.requires_grad = True
        
        softmax, _ = model(pts)
        org_pred = softmax.data.max(1)[1]
        loss = F.nll_loss(softmax, org_pred)
        model.zero_grad()
        loss.backward()
        pts_grad = pts.grad.data
        pts_adv = pts + eps * pts_grad.sign()

        chamfer_loss = num(chamfer_batch(pts_adv.transpose(2, 1), pts.transpose(2, 1)))
        softmax, _ = model(pts_adv)
        adv_pred = softmax.data.max(1)[1]

        not_org_label = num((adv_pred == org_pred).double().mean())
        log.info(eps,not_org_label=not_org_label, chamfer = chamfer_loss)
        break
    #disp_pt_cloud(num(pts_adv.detach().transpose(2, 1)[1]))

0.00000 not_org_label: 1.0 chamfer: 0.0 
0.00526 not_org_label: 0.875 chamfer: 6.1631727 
0.01053 not_org_label: 0.625 chamfer: 12.032362 
0.01579 not_org_label: 0.625 chamfer: 16.966932 
0.02105 not_org_label: 0.625 chamfer: 20.560429 
0.02632 not_org_label: 0.625 chamfer: 23.336508 
0.03158 not_org_label: 0.625 chamfer: 25.889557 
0.03684 not_org_label: 0.625 chamfer: 28.534967 
0.04211 not_org_label: 0.625 chamfer: 31.30495 
0.04737 not_org_label: 0.625 chamfer: 34.115204 
0.05263 not_org_label: 0.625 chamfer: 36.873405 
0.05789 not_org_label: 0.875 chamfer: 39.57132 
0.06316 not_org_label: 0.875 chamfer: 42.243492 
0.06842 not_org_label: 0.875 chamfer: 44.895683 
0.07368 not_org_label: 0.875 chamfer: 47.56867 
0.07895 not_org_label: 0.875 chamfer: 50.247284 
0.08421 not_org_label: 0.875 chamfer: 52.903633 
0.08947 not_org_label: 0.875 chamfer: 55.57978 
0.09474 not_org_label: 0.875 chamfer: 58.269615 
0.10000 not_org_label: 0.75 chamfer: 60.985542 
